In [1]:
import OpenEXR
import Imath
import numpy as np
from PIL import Image

In [10]:
def exr_to_image(exr_path, output_path):
    """
    读取EXR文件并将其转换为标准图像格式（如PNG）。

    :param exr_path: 输入的.exr文件路径。
    :param output_path: 输出的图像文件路径（例如 'output.png'）。
    """
    try:
        # 1. 读取EXR文件
        exr_file = OpenEXR.InputFile(exr_path)
        
        # 2. 获取文件头信息
        header = exr_file.header()
        dw = header['dataWindow']
        width = dw.max.x - dw.min.x + 1
        height = dw.max.y - dw.min.y + 1

        # 3. 获取通道和数据类型
        # EXR可以是半浮点(16bit)或单浮点(32bit)
        # 我们需要根据文件类型来正确解析
        pt = Imath.PixelType(Imath.PixelType.FLOAT) # 假设为32位浮点数
        if header['channels']['R'].type == Imath.PixelType(Imath.PixelType.HALF):
            pt = Imath.PixelType(Imath.PixelType.HALF) # 如果是16位半浮点数

        # 4. 读取RGB通道数据
        red_str = exr_file.channel('R', pt)
        green_str = exr_file.channel('G', pt)
        blue_str = exr_file.channel('B', pt)

        # 5. 将字节数据转换为NumPy数组
        # 根据数据类型(float16或float32)转换
        dtype = np.float16 if pt == Imath.PixelType(Imath.PixelType.HALF) else np.float32
        red = np.frombuffer(red_str, dtype=dtype).reshape(height, width)
        green = np.frombuffer(green_str, dtype=dtype).reshape(height, width)
        blue = np.frombuffer(blue_str, dtype=dtype).reshape(height, width)

        # 6. 将三个通道合并成一个RGB图像数组
        # 此时图像数据是浮点数，范围可能远大于[0, 1]
        hdr_image = np.dstack((red, green, blue))
        
        # --- HDR到LDR的关键转换步骤 ---
        # 7. 色调映射/范围压缩
        # 方法A: 简单的线性裁剪 (可能会丢失高光细节)
        # ldr_image = np.clip(hdr_image, 0, 1)

        # 方法B: 应用Gamma校正 (一种简单的色调映射)
        # 这是为了将线性空间的HDR数据转换到sRGB色彩空间
        gamma = 2.2
        ldr_image = np.power(hdr_image, 1.0/gamma)

        # 确保数据仍在0-1范围内
        ldr_image = np.clip(ldr_image, 0, 1)

        # 8. 将浮点数数组[0, 1]转换为8位整数数组[0, 255]
        image_8bit = (ldr_image * 255).astype(np.uint8)

        # 9. 使用Pillow保存为PNG或JPG
        pil_image = Image.fromarray(image_8bit)
        pil_image.save(output_path)
        print(f"成功将 {exr_path} 转换为 {output_path}")

    except Exception as e:
        print(f"处理文件时出错: {e}")

In [11]:
#exr_to_image('C:\\Users\\11020\\Downloads\\32k.exr', 'C:\\Users\\11020\\Downloads\\32k.png')
exr_to_image('C:\\Users\\11020\\Downloads\\32k.exr', 'C:\\Users\\11020\\Downloads\\32k.jpg')

成功将 C:\Users\11020\Downloads\32k.exr 转换为 C:\Users\11020\Downloads\32k.jpg
